In [13]:
from __future__ import print_function, unicode_literals
from builtins import open
from future import standard_library
standard_library.install_aliases()

import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import nipype.interfaces.utility as util
import nipype.interfaces.ants as ants
import nipype.interfaces.io as io
import nipype.pipeline.engine as pe

from niflow.nipype1.workflows.smri.ants import ANTSTemplateBuildSingleIterationWF

In [24]:
base_dir = "/mnt/HDD/Data/nipype_test/"
data_dir = os.path.join(base_dir, "samples")
data_paths = [os.path.join(data_dir, name) for name in os.listdir(data_dir) if name[-4:] == '.nii']
input_images = np.random.choice(data_paths, 3).tolist()

In [25]:
tbuilder = pe.Workflow(name="ANTSTemplateBuilder")
tbuilder.base_dir = data_dir

In [26]:
datasource = pe.Node(
    interface=util.IdentityInterface(fields=['imageList', 'passiveImagesDictionariesList']),
    run_without_submitting=True,
    name='InputImages'
)
datasource.inputs.imageList = input_images
datasource.inputs.passiveImagesDictionariesList = input_images
datasource.inputs.sort_filelist = True

In [27]:
initAvg = pe.Node(interface=ants.AverageImages(), name='initAvg')
initAvg.inputs.dimension = 3
initAvg.inputs.normalize = True

In [28]:
tbuilder.connect(datasource, "imageList", initAvg, "images")

In [29]:
buildTemplateIteration1 = ANTSTemplateBuildSingleIterationWF('iteration01')

tbuilder.connect(
    initAvg, 'output_average_image', 
    buildTemplateIteration1, 'inputspec.fixed_image'
)

tbuilder.connect(
    datasource, 'imageList', 
    buildTemplateIteration1,'inputspec.images'
)

tbuilder.connect(
    datasource, 'passiveImagesDictionariesList',
    buildTemplateIteration1, 'inputspec.ListOfPassiveImagesDictionaries'
)

In [30]:
buildTemplateIteration2 = ANTSTemplateBuildSingleIterationWF('iteration02')

tbuilder.connect(
    buildTemplateIteration1, 'outputspec.template',
    buildTemplateIteration2, 'inputspec.fixed_image'
)

tbuilder.connect(
    datasource, 'imageList', 
    buildTemplateIteration2, 'inputspec.images'
)

tbuilder.connect(
    datasource, 'passiveImagesDictionariesList', 
    buildTemplateIteration2,'inputspec.ListOfPassiveImagesDictionaries'
)

In [31]:
datasink = pe.Node(io.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.join(base_dir, "results")

tbuilder.connect(buildTemplateIteration2, 'outputspec.template', datasink,
                 'PrimaryTemplate')
tbuilder.connect(buildTemplateIteration2,
                 'outputspec.passive_deformed_templates', datasink,
                 'PassiveTemplate')
tbuilder.connect(initAvg, 'output_average_image', datasink,
                 'PreRegisterAverage')

In [32]:
tbuilder.run()

210328-13:48:09,995 nipype.workflow INFO:
	 Workflow ANTSTemplateBuilder settings: ['check', 'execution', 'logging', 'monitoring']
210328-13:48:10,62 nipype.workflow INFO:
	 Running serially.
210328-13:48:10,63 nipype.workflow INFO:
	 [Node] Setting-up "ANTSTemplateBuilder.initAvg" in "/mnt/HDD/Data/nipype_test/samples/ANTSTemplateBuilder/initAvg".
210328-13:48:10,70 nipype.workflow INFO:
	 [Node] Running "initAvg" ("nipype.interfaces.ants.utils.AverageImages"), a CommandLine Interface with command:
AverageImages 3 average.nii 1 /mnt/HDD/Data/nipype_test/samples/ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001114556321_S13408_I118671.nii /mnt/HDD/Data/nipype_test/samples/ADNI_003_S_1021_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070918111516027_S21771_I73506.nii /mnt/HDD/Data/nipype_test/samples/ADNI_002_S_1268_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070802165115484_S26498_I64037.nii
210328-13:48:10,439 nipype.interface INFO:
	 stdout 2021-03-2

RuntimeError: Workflow did not execute cleanly. Check log for details